In [192]:
import censusdata
import psycopg2
import pandas as pd
import csv
from io import StringIO 

# define state and county
geo = censusdata.censusgeo([('state', '17'), ('county', '031'), ('block group', '*')])

# choose variables of interest
variables = ['B23025_001E', 'B23025_002E', 'B23025_003E', 'B23025_004E', 'B23025_005E', 'B23025_006E', 'B23025_007E']


In [193]:
# extract data
data = censusdata.download('acs5', 2015, geo, variables)

# rename variable names
data.rename({'B23025_001E': 'est_total', 
             'B23025_002E': 'est_total_in_labor_force', 
             'B23025_003E': 'est_total_in_labor_force_civilian', 
             'B23025_004E': 'est_total_in_labor_force_civilian_employed', 
             'B23025_005E': 'est_total_in_labor_force_civilian_unemployed', 
             'B23025_006E': 'est_total_in_labor_force_armed_forces', 
             'B23025_007E': 'est_total_not_in_labor_force'}, 
            axis = 1, inplace = True)

# name index column
data.index.name = 'geo_info'

# reset index column to normal column
data.reset_index(inplace = True)

# data cleaning
data['geo_info'] = data['geo_info'].astype(str)
data['geo_info'] = data['geo_info'].str.split(", ").tolist()


data['state'] = data['geo_info'].apply(lambda x: x[3])
data['state'] = data['state'].str.split(":").tolist()
data['state'] = data['state'].apply(lambda x: x[0])

data['county'] = data['geo_info'].apply(lambda x: x[2])

data['block_group'] = data['geo_info'].apply(lambda x: x[0])

data = data.drop(columns = 'geo_info')


In [194]:
data.to_csv(r'./etl_data.csv', index = False)

In [195]:
conn = psycopg2.connect(
    host='acs-db.mlpolicylab.dssg.io',
    port = '5432',
    user = 'mlpp_student',
    password = 'CARE-horse-most',
    database = 'acs_data_loading'
)

cur = conn.cursor()

cur.execute(f"CREATE TABLE acs.yunhanx_acs_data(\
           est_total INTEGER,\
           est_total_in_labor_force INTEGER,\
           est_total_in_labor_force_civilian INTEGER,\
           est_total_in_labor_force_civilian_employed INTEGER,\
           est_total_in_labor_force_civilian_unemployed INTEGER,\
           est_total_in_labor_force_armed_forces INTEGER,\
           est_total_not_in_labor_force INTEGER,\
           state VARCHAR,\
           county VARCHAR,\
           block_group VARCHAR)")

conn.commit()
cur.close()
conn.close()

In [203]:
conn = psycopg2.connect(
    host='acs-db.mlpolicylab.dssg.io',
    port = '5432',
    user = 'mlpp_student',
    password = 'CARE-horse-most',
    database = 'acs_data_loading'
)


sio = StringIO()
sio.write(data.to_csv(index = None, header = None))
sio.seek(0)

with conn.cursor() as c:
    c.execute('SET search_path TO acs')
    c.copy_from(
    file = sio,
    table = "yunhanx_acs_data",
    columns = ["est_total", 
               "est_total_in_labor_force", 
               "est_total_in_labor_force_civilian",
               "est_total_in_labor_force_civilian_employed",
               "est_total_in_labor_force_civilian_unemployed",
               "est_total_in_labor_force_armed_forces",
               "est_total_not_in_labor_force",
               "state",
               "county",
               "block_group"],
    sep = ","
    
)

conn.commit()
conn.close()